In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Inisialisasi splitter untuk memisahkan dokumen
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# Inisialisasi variabel untuk menampung dokumen yang digabung
combined_docs = []

# Fungsi untuk memuat dan memisahkan dokumen
def load_and_split_pdf(file_path, text_splitter):
    loader = PyPDFLoader(file_path)
    data = loader.load()  # Muat dokumen sebagai satu kesatuan
    split_docs = text_splitter.split_documents(data)  # Pisahkan dokumen
    return split_docs

# Daftar file yang akan diproses
file_paths = [
    "5. UU-40-2007 PERSEROAN TERBATAS.pdf",
    "e39ab-uu-nomor-8-tahun-1999.pdf",
    "kolonial_kuh_perdata_fix.pdf",
    "KUH DAGANG.pdf",
    "UU Nomor  19 Tahun 2016.pdf",
    "UU Nomor 13 Tahun 2003.pdf",
    "UU_1999_30.pdf",
    "UU_Nomor_11_Tahun_2020-compressed.pdf",
]

base = "/Docs/"
# Proses setiap dokumen secara otomatis
for file_path in file_paths:
    full_path = base_path + file_path  # Gabungkan base path dengan file path
    print(f"Processing: {full_path}")
    additional_docs = load_and_split_pdf(full_path, text_splitter)  # Load dan split dokumen
    combined_docs += additional_docs  # Gabungkan ke dalam combined_docs
    print(f"Added {len(additional_docs)} documents from {file_path}")

# Total dokumen setelah penggabungan
print("Total number of combined documents: ", len(combined_docs))

Processing: 5. UU-40-2007 PERSEROAN TERBATAS.pdf
Added 280 documents from 5. UU-40-2007 PERSEROAN TERBATAS.pdf
Processing: e39ab-uu-nomor-8-tahun-1999.pdf
Added 87 documents from e39ab-uu-nomor-8-tahun-1999.pdf
Processing: kolonial_kuh_perdata_fix.pdf
Added 1150 documents from kolonial_kuh_perdata_fix.pdf
Processing: KUH DAGANG.pdf
Added 604 documents from KUH DAGANG.pdf
Processing: UU Nomor  19 Tahun 2016.pdf
Added 44 documents from UU Nomor  19 Tahun 2016.pdf
Processing: UU Nomor 13 Tahun 2003.pdf
Added 242 documents from UU Nomor 13 Tahun 2003.pdf
Processing: UU_1999_30.pdf
Added 84 documents from UU_1999_30.pdf
Processing: UU_Nomor_11_Tahun_2020-compressed.pdf
Added 2090 documents from UU_Nomor_11_Tahun_2020-compressed.pdf
Total number of combined documents:  4581


In [3]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 


c:\Kuliah\Legalink\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,               # Sesuai dengan generation_config
    top_p=0.95,                    # Menambahkan top_p
    top_k=40,                      # Menambahkan top_k
    max_tokens=8192,               # Menyesuaikan max_output_tokens
    response_mime_type="text/plain" # Menyesuaikan response_mime_type
)


In [20]:
# Define system prompt with context placeholder
system_prompt = """
Anda adalah AI bisnis yang sangat terlatih (berbasis Gemini), bertugas membantu pengguna menganalisis ide bisnis mereka dengan memberikan saran, checklist, dan rekomendasi hukum yang relevan. Berikut adalah informasi dokumen yang berhasil diambil dari sistem (RAG):

{context}

Gunakan informasi di atas **hanya jika relevan** dengan pertanyaan pengguna. Jika tidak relevan, abaikan konten tersebut.

### Instruksi Tugas

1. **Analisis Informasi Bisnis**:
   - Pahami detail ide bisnis yang disampaikan pengguna (seperti model usaha, target pasar, bentuk usaha, lokasi operasional, regulasi diketahui, dan lain-lain).
   - Hubungkan dengan dokumen RAG di atas, jika relevan.

2. **Menyusun Checklist**:
   - Buat daftar poin penting yang harus diperhatikan (pendirian usaha, izin, legalitas produk, ketentuan pemasaran, dll.).
   - Tuliskan deskripsi singkat untuk setiap poin checklist.
   - Jika dokumen RAG menyinggung pasal atau regulasi tertentu, berikan "Acuan Pasal" agar pengguna tahu sumber hukumnya.

3. **Identifikasi Risiko dan Rekomendasi**:
   - Temukan potensi risiko hukum atau administratif yang mungkin muncul.
   - Berikan saran untuk mengurangi risiko tersebut (mis. klausa perjanjian, tata cara pendaftaran merek, dsb.).

4. **Format Jawaban**:
Silakan jawab dalam format JSON berikut 
   - Isi atau kosongkan bagian yang tidak relevan.
   - Pastikan struktur JSON valid.

5. **Gaya Bahasa**:
   - Gunakan nada profesional tetapi ramah dan mudah dipahami.
   - Hindari penggunaan istilah hukum berlebihan; jika perlu, berikan penjelasan singkat.

**Catatan Penting**:
- Jika dokumen RAG tidak berisi informasi hukum tertentu, jangan mengada-ada (pastikan data akurat).
- Kualitas jawaban harus tetap didahulukan: relevan, tepat, dan jelas.

"""

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv



load_dotenv() 


True

In [14]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = '''Anda adalah AI bisnis yang sangat terlatih (berbasis Gemini) yang ditugaskan untuk memahami dan membantu menganalisis ide bisnis. Tugas Anda saat ini adalah:

1. **Memeriksa Kebutuhan Konteks**:
   - Periksa apakah pertanyaan terbaru pengguna merujuk pada informasi yang muncul di percakapan (chat history) sebelumnya.
   - Jika ya, gabungkan informasi tersebut ke dalam pertanyaan agar pertanyaan menjadi utuh (mandiri).

2. **Reformulasi Pertanyaan**:
   - Apabila pertanyaan tidak lengkap tanpa konteks sebelumnya, reformulasikan agar menjadi mandiri dan jelas.
   - Jika pertanyaan sudah mandiri, kembalikan apa adanya tanpa perubahan.

3. **Output Hanya Pertanyaan**:
   - Jangan memberikan jawaban, penjelasan tambahan, analisis, atau komentar lain.
   - Hanya kembalikan pertanyaan yang sudah direvisi sehingga siap diproses lebih lanjut oleh sistem.

**Contoh**:
- Chat History:
  - Q1: "Ide bisnis saya adalah snack sehat. Apa langkah awalnya?"
  - A1: "Langkah awal adalah menyiapkan legalitas usaha."
- Pertanyaan Baru: "Bagaimana cara mendaftarkan merek?"
  - **Reformulasi**: "Bagaimana cara mendaftarkan merek untuk bisnis snack sehat?"

Jangan menyertakan kata-kata seperti “Reformulasi” atau “A1,” cukup kembalikan pertanyaan akhir. 

'''

vectorstore = Chroma.from_documents(documents=combined_docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
    
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []


In [23]:
business_info = {
    "Nama Ide Bisnis": "Snack Sehat Nusantara",
    "Deskripsi Ide": "Menjual snack sehat berbahan lokal seperti singkong dan talas dengan berbagai rasa.",
    "Target Pasar": "Anak muda usia 18-35 di kota besar.",
    "Model Bisnis": "Penjualan langsung dan online melalui e-commerce.",
    "Lokasi Operasional": "Jakarta",
    "Bentuk Usaha": "PT",
    "Kebutuhan Modal": "100 juta",
    "Tim atau Pendiri": ["Fani Najmun Nisa", "Rizky Mahardika"],
    "Jenis Produk/Jasa": "Makanan ringan",
    "Rencana Pemasaran": "Menggunakan media sosial untuk promosi dan influencer marketing.",
    "Regulasi yang Diketahui": ["UU Perlindungan Konsumen", "Izin PIRT"],
    "Kemitraan/Investor": "Tidak ada"
}

prompt = f"""
Saya memiliki ide bisnis sebagai berikut:
- Nama Ide Bisnis: {business_info.get('Nama Ide Bisnis', 'Tidak disebutkan')}
- Deskripsi Ide: {business_info.get('Deskripsi Ide', 'Tidak disebutkan')}
- Target Pasar: {business_info.get('Target Pasar', 'Tidak disebutkan')}
- Model Bisnis: {business_info.get('Model Bisnis', 'Tidak disebutkan')}
- Lokasi Operasional: {business_info.get('Lokasi Operasional', 'Tidak disebutkan')}
- Bentuk Usaha: {business_info.get('Bentuk Usaha', 'Tidak disebutkan')}
- Kebutuhan Modal: {business_info.get('Kebutuhan Modal', 'Tidak disebutkan')}
- Tim atau Pendiri: {', '.join(business_info.get('Tim atau Pendiri', ['Tidak disebutkan']))}
- Jenis Produk/Jasa: {business_info.get('Jenis Produk/Jasa', 'Tidak disebutkan')}
- Rencana Pemasaran: {business_info.get('Rencana Pemasaran', 'Tidak disebutkan')}
- Regulasi yang Diketahui: {', '.join(business_info.get('Regulasi yang Diketahui', ['Tidak disebutkan']))}
- Kemitraan/Investor: {business_info.get('Kemitraan/Investor', 'Tidak disebutkan')}

Tolong bantu analisis ide bisnis saya. Berikut permintaan saya:

1. Analisis singkat terhadap informasi bisnis di atas.
2. Susun checklist poin-poin penting yang harus saya perhatikan, serta sertakan acuan pasal dari RAG (jika relevan).
3. Jelaskan risiko hukum atau administratif yang mungkin muncul.
4. Berikan rekomendasi atau langkah tambahan untuk memperkuat rencana bisnis saya.

Jangan lupa, jawaban akhir harus dalam format JSON sesuai instruksi yang telah ditetapkan.
"""


In [24]:
ai_msg = rag_chain.invoke({"input": prompt, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=prompt),
        AIMessage(content=ai_msg["answer"]),
    ]
)
print(ai_msg["answer"])
print()

```json
{
  "analisis_informasi_bisnis": "Snack Sehat Nusantara memiliki potensi pasar yang baik dengan menyasar anak muda di kota besar yang cenderung peduli kesehatan. Model bisnis penjualan langsung dan online efektif menjangkau target pasar.  Penting untuk memastikan kualitas bahan baku dan rasa produk agar dapat bersaing. Modal 100 juta perlu dialokasikan secara efektif untuk operasional, pemasaran, dan perizinan.  Strategi pemasaran dengan media sosial dan influencer marketing cukup relevan dengan target pasar.",
  "checklist": [
    {
      "poin": "Pendirian PT",
      "deskripsi": "Menyiapkan dokumen-dokumen pendirian PT seperti akta pendirian, NPWP, SIUP, dan TDP. Pastikan memenuhi persyaratan modal dan susunan pengurus.",
      "acuan_pasal": ""
    },
    {
      "poin": "Izin PIRT",
      "deskripsi": "Mengurus Izin PIRT (Pangan Industri Rumah Tangga) dari Dinas Kesehatan setempat. Izin ini wajib untuk produk makanan dan minuman yang diproduksi oleh usaha kecil dan menenga

In [25]:
while (True) : 
    question = input("Masukan Input : ")
    if question =="stop": break
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=question),
            AIMessage(content=ai_msg["answer"]),
        ]
    )
    print(ai_msg["answer"])
    print()

```json
{
  "analisis_informasi_bisnis": "Pertanyaan Anda menunjukkan pemahaman yang baik tentang aspek hukum dan administratif dalam menjalankan bisnis makanan. Klarifikasi poin-poin ini penting untuk mencegah masalah di kemudian hari.",
  "checklist": [
    {
      "poin": "Pendirian PT",
      "deskripsi": "Dokumen yang umumnya diperlukan antara lain: Akta Pendirian yang dibuat oleh Notaris, Surat Keterangan Domisili Perusahaan, NPWP, Surat Izin Usaha Perdagangan (SIUP), Tanda Daftar Perusahaan (TDP), dan pengurusan NIB melalui OSS.",
      "acuan_pasal": ""
    },
    {
      "poin": "Izin PIRT",
      "deskripsi": "Pengurusan Izin PIRT umumnya dilakukan di Dinas Kesehatan tingkat kabupaten/kota. Namun, tetap perhatikan regulasi terbaru dan persyaratan khusus yang mungkin berlaku di daerah operasional Anda (Jakarta). Pastikan juga produk Anda memenuhi standar keamanan pangan yang ditetapkan BPOM.",
      "acuan_pasal": ""
    },
    {
      "poin": "Registrasi Merek Dagang",
      

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [26]:
while (True) : 
    question = input("Masukan Input : ")
    if question =="stop": break
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=question),
            AIMessage(content=ai_msg["answer"]),
        ]
    )
    print(ai_msg["answer"])
    print()

```json
{
  "analisis_informasi_bisnis": "Pertanyaan Anda menunjukkan fokus yang baik pada mitigasi risiko hukum. Memahami langkah-langkah hukum yang dapat diambil dalam situasi-situasi tersebut sangat penting untuk melindungi bisnis Anda.",
  "checklist": [],
  "risiko": [
    {
      "deskripsi": "Pelanggaran HAKI: Peniruan merek dagang atau kemasan.",
      "rekomendasi": "Langkah hukum:  Mengirimkan somasi,  mediasi,  arbitrase,  atau mengajukan gugatan ke Pengadilan Niaga.  Anda dapat menuntut ganti rugi dan penghentian penggunaan merek/desain yang melanggar HAKI."
    },
    {
      "deskripsi": "Sengketa dengan Supplier: Keterlambatan pengiriman.",
      "rekomendasi": "Hukum Indonesia melalui Kitab Undang-Undang Hukum Perdata (KUHPerdata) mengatur tentang wanprestasi.  Anda dapat menuntut pemenuhan kewajiban, ganti rugi, atau pembatalan perjanjian.  Klausul denda dalam perjanjian juga dapat diaktifkan."
    },
    {
      "deskripsi": "Keamanan Pangan: Keracunan makanan.",
    

In [27]:
while (True) : 
    question = input("Masukan Input : ")
    if question =="stop": break
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=question),
            AIMessage(content=ai_msg["answer"]),
        ]
    )
    print(ai_msg["answer"])
    print()

```json
{
  "analisis_informasi_bisnis": "Ya, bisnis makanan ringan seperti 'Snack Sehat Nusantara' **wajib** mencantumkan informasi komposisi bahan dan tanggal kedaluwarsa pada label produk.  Ini bukan hanya praktik bisnis yang baik, tetapi juga kewajiban hukum.",
  "checklist": [],
  "risiko": [],
  "rekomendasi_tambahan": [
    {
      "deskripsi": "Kewajiban Pencantuman Label",
      "rekomendasi": "Informasi komposisi bahan dan tanggal kedaluwarsa wajib dicantumkan pada label makanan. Hal ini diatur dalam berbagai peraturan perundang-undangan di Indonesia, termasuk Undang-Undang Nomor 8 Tahun 1999 tentang Perlindungan Konsumen dan Peraturan Badan Pengawas Obat dan Makanan (BPOM) terkait label pangan."
    },
    {
      "deskripsi": "Konsekuensi Hukum",
      "rekomendasi": "Jika informasi tersebut tidak dicantumkan, konsekuensi hukumnya dapat berupa sanksi administratif seperti peringatan, teguran tertulis, denda, penarikan produk dari peredaran, hingga pencabutan izin usaha.  Da